In [1]:
import os
import rasterio
from rasterio.windows import Window
import geopandas as gpd
from shapely.geometry import box
from ultralytics import YOLO
import numpy as np

In [2]:
# Configurações
image_path = 'dados/DJI_0128.tif'  # Caminho do GeoTIFF original
preproc_dir = 'preproc'  # Diretório para salvar os tiles
geojson_output_path = 'resultado/DJI_0128_deteccao_pessoas.geojson'  # Saída final
tile_size = 1280  # Tamanho dos tiles (1280x1280)

# Criar o diretório para tiles, se não existir
os.makedirs(preproc_dir, exist_ok=True)

crs = "EPSG:4326"

In [3]:
# Dividir o GeoTIFF em tiles menores
with rasterio.open(image_path) as src:
    transform = src.transform
    crs = src.crs
    width = src.width
    height = src.height

    # Iterar sobre a imagem para criar tiles
    for i in range(0, width, tile_size):
        for j in range(0, height, tile_size):
            window = Window(i, j, min(tile_size, width - i), min(tile_size, height - j))
            transform_tile = src.window_transform(window)
            tile_path = os.path.join(preproc_dir, f"tile_{i}_{j}.tif")

            # Salvar o tile como GeoTIFF
            with rasterio.open(
                tile_path,
                'w',
                driver='GTiff',
                height=window.height,
                width=window.width,
                count=src.count,
                dtype=src.dtypes[0],
                crs=crs,
                transform=transform_tile,
            ) as dst:
                dst.write(src.read(window=window))

            print(f"Tile salvo: {tile_path}")

Tile salvo: preproc\tile_0_0.tif
Tile salvo: preproc\tile_0_1280.tif
Tile salvo: preproc\tile_0_2560.tif
Tile salvo: preproc\tile_0_3840.tif
Tile salvo: preproc\tile_0_5120.tif
Tile salvo: preproc\tile_0_6400.tif
Tile salvo: preproc\tile_1280_0.tif
Tile salvo: preproc\tile_1280_1280.tif
Tile salvo: preproc\tile_1280_2560.tif
Tile salvo: preproc\tile_1280_3840.tif
Tile salvo: preproc\tile_1280_5120.tif
Tile salvo: preproc\tile_1280_6400.tif
Tile salvo: preproc\tile_2560_0.tif
Tile salvo: preproc\tile_2560_1280.tif
Tile salvo: preproc\tile_2560_2560.tif
Tile salvo: preproc\tile_2560_3840.tif
Tile salvo: preproc\tile_2560_5120.tif
Tile salvo: preproc\tile_2560_6400.tif
Tile salvo: preproc\tile_3840_0.tif
Tile salvo: preproc\tile_3840_1280.tif
Tile salvo: preproc\tile_3840_2560.tif
Tile salvo: preproc\tile_3840_3840.tif
Tile salvo: preproc\tile_3840_5120.tif
Tile salvo: preproc\tile_3840_6400.tif
Tile salvo: preproc\tile_5120_0.tif
Tile salvo: preproc\tile_5120_1280.tif
Tile salvo: preproc

In [26]:
# Carregar o modelo YOLO11
model = YOLO('yolo11n.pt')

# Processar os tiles e consolidar resultados
geo_boxes = []
scores = []

for tile_file in os.listdir(preproc_dir):
    if tile_file.endswith('.tif'):
        tile_path = os.path.join(preproc_dir, tile_file)

        with rasterio.open(tile_path) as src:
            image = src.read([1, 2, 3])  # Supondo que seja RGB (3 bandas)
            image = np.transpose(image, (1, 2, 0))  # Reorganizar para altura x largura x canais
            image = (image - image.min()) / (image.max() - image.min()) * 255  # Normalizar para [0, 255]
            image = image.astype(np.uint8)  # Garantir tipo uint8
            transform_tile = src.transform

        # Realizar detecção no tile
        results = model.predict(image, classes=[0], conf=0.05, iou=0.1, verbose=True)  # Classe 0 é "person"

        for result in results[0].boxes.data.tolist():
            x_min, y_min, x_max, y_max, confidence, *extra = result

            # Converter coordenadas de pixel do tile para coordenadas globais
            top_left = rasterio.transform.xy(transform_tile, int(y_min), int(x_min), offset="ul")
            bottom_right = rasterio.transform.xy(transform_tile, int(y_max), int(x_max), offset="ul")

            # Criar uma caixa geográfica com coordenadas globais
            geo_box = box(top_left[0], bottom_right[1], bottom_right[0], top_left[1])
            geo_boxes.append(geo_box)
            scores.append(confidence)

C:\Users\jac\AppData\Local\Temp\ipykernel_16996\659644112.py:15: RuntimeWarning: invalid value encountered in divide
  image = (image - image.min()) / (image.max() - image.min()) * 255  # Normalizar para [0, 255]
C:\Users\jac\AppData\Local\Temp\ipykernel_16996\659644112.py:16: RuntimeWarning: invalid value encountered in cast
  image = image.astype(np.uint8)  # Garantir tipo uint8



0: 640x640 (no detections), 17.0ms
Speed: 13.0ms preprocess, 17.0ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 persons, 6.0ms
Speed: 26.3ms preprocess, 6.0ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 9.8ms
Speed: 23.7ms preprocess, 9.8ms inference, 8.8ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 17.0ms
Speed: 14.2ms preprocess, 17.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 14.9ms
Speed: 17.5ms preprocess, 14.9ms inference, 10.5ms postprocess per image at shape (1, 3, 640, 640)

0: 128x640 (no detections), 16.4ms
Speed: 0.0ms preprocess, 16.4ms inference, 0.0ms postprocess per image at shape (1, 3, 128, 640)

0: 640x640 8 persons, 33.4ms
Speed: 0.0ms preprocess, 33.4ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 12 persons, 18.5ms
Speed: 13.0ms preprocess, 18.5ms inference, 3.1m

In [27]:
# Criar um GeoDataFrame consolidado
gdf = gpd.GeoDataFrame({"score": scores}, geometry=geo_boxes, crs=crs)

# Calcular áreas dos polígonos e filtrar
# gdf.area assume que o CRS está em metros; verifique e converta se necessário
if gdf.crs and gdf.crs.to_string() != "EPSG:4326":  # CRS projetado
    gdf = gdf[gdf.geometry.area <= 3.0]
else:  # CRS geográfico, precisa projetar para calcular a área corretamente
    gdf = gdf.to_crs("EPSG:3857")  # Converter para CRS métrico
    gdf = gdf[gdf.geometry.area <= 3.0]
    gdf = gdf.to_crs(crs)  # Retornar ao CRS original

# Salvar o resultado consolidado como GeoJSON
gdf.to_file(geojson_output_path, driver="GeoJSON")

print(f"GeoJSON consolidado salvo em {geojson_output_path}")

GeoJSON consolidado salvo em resultado/DJI_0128_deteccao_pessoas.geojson
